# Libraries

In [3]:
conda install Scripy

import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import mpl, plt
import Scripy

SyntaxError: invalid syntax (3455506112.py, line 1)

# Adding Assets and pulling historical data


In [ ]:
stocks=['AAPL','TSLA','MCD','KO','TMO','MMM','NKE']

eD=dt.datetime.today()
sD=eD-dt.timedelta(2*365)

price=yf.download(stocks,
                    start=sD,
                    end=eD)['Adj Close']
returns=price.pct_change().dropna()

# Calculate the mean, variance, and correlation matrix for all assets

In [ ]:
class Tools:
    def __init__(self,returns, assets):
        self.returns = returns
        self.assets = assets
 
    def simpleComp(self):
        self.tbl=pd.DataFrame(index=self.assets)
        self.tbl['Mean']=returns.mean()
        self.tbl['Variance']=returns.var()
        return(self.tbl.T)
   
    def correlation(self):
        self.tbl=pd.DataFrame(index=self.assets)
        self.corr=returns.corr().round(3)
        return(self.corr)

In [ ]:
tls=Tools(returns=returns, assets=stocks)
tbl=tls.simpleComp()
corr=tls.correlation()
display(tbl,corr)

# Merge Assets into Portfolio

### ensure random weights equal 100%

In [ ]:
weights = np.random.random(len(stocks))
weights /= np.sum(weights)
weights

In [ ]:
weights.sum()

### now to annualize the portfolio returns based of randomized weights using the monte carlo simulation


In [ ]:
#portfolio returns
def pRet(weights):
    return np.sum(returns.mean() * weights) * 252
#portfolio volumes
def pVol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252,weights)))

retCol = []
volCol = []

for p in range (1000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    retCol.append(pRet(weights))
    volCol.append(pVol(weights))

retCol = np.array(retCol)
volCol = np.array(volCol)

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x: port_ret(x) - tret},
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in weights)
trets = np.linspace(0.05, 0.2, 50)
tvols = []
for tret in trets:
    res = sco.minimize(port_vol, eweights, method='SLSQP',
                       bounds=bnds, constraints=cons)
    tvols.append(res['fun'])
tvols = np.array(tvols)